In [1]:
import gymnasium

In [2]:
num_envs = 64
env_id = "RlTrader-v0" 
seed = 0
learning_rate=2.5e-4

In [3]:
from typing import Optional
import numpy as np
import torch as th
from packaging import version
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import VecEnvWrapper, VecMonitor
from stable_baselines3.common.vec_env.base_vec_env import (
  VecEnvObs,
  VecEnvStepReturn,
)

import litepool
from litepool.python.protocol import LitePool

2024-07-19 23:09:33.129068: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-19 23:09:33.157961: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 23:09:33.157985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 23:09:33.158705: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-19 23:09:33.165538: I tensorflow/core/platform/cpu_feature_guar

In [4]:
class VecAdapter(VecEnvWrapper):
  def __init__(self, venv: LitePool):
    venv.num_envs = venv.spec.config.num_envs
    super().__init__(venv=venv)
    self.env_ids = None

  def step_async(self, actions: np.ndarray) -> None:
    self.actions = actions

  def reset(self) -> VecEnvObs:
      self.env_ids = None
      return self.venv.reset()[0]

  def seed(self, seed: Optional[int] = None) -> None:
    pass

  def step_wait(self) -> VecEnvStepReturn:
      if self.env_ids is None:
          self.venv.send(self.actions)
      else:
          self.venv.send(self.actions, self.env_ids)
    
      obs, rewards, terms, truncs, info_dict = self.venv.recv()
      dones = terms + truncs
      infos = []
      for i in range(self.num_envs):
          infos.append({
              key: info_dict[key][i]
              for key in info_dict.keys()
              if isinstance(info_dict[key], np.ndarray)
          })
      if dones[i]:
          print(rewards[i])
          infos[i]["terminal_observation"] = obs[i]
          obs[i] = self.venv.reset(np.array([i]))[0]
      return obs, rewards, dones, infos

In [5]:
env = env = litepool.make("RlTrader-v0", env_type="gymnasium", 
                          num_envs=num_envs, batch_size=4, 
                          num_threads=16, 
                          filename="deribit.csv", 
                          balance=0.1,
                          depth=20)
env.spec.id = env_id
env = VecAdapter(env)
env = VecMonitor(env)

/home/pravin/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:77: UserWarning: The `render_mode` attribute is not defined in your environment. It will be set to None.
  warnings.warn("The `render_mode` attribute is not defined in your environment. It will be set to None.")


In [ ]:
kwargs = dict(use_sde=True, sde_sample_freq=4)

model = PPO(
  "MlpPolicy",
  env,
  n_steps=1024,
  learning_rate=1e-3,
  gae_lambda=0.95,
  gamma=0.9,
  verbose=1,
  seed=seed,
  **kwargs
)

model.learn(100)

Using cpu device
